# Terminology:

- An **experiment** is a (possibly long-lived) sequence of measurements, captured in many DataSets, performed on a single sample, in a single fridge. The user tells us when a new experiment is started. Experiments have a (user-provided) name.
- A **run** is a single sweep or other measurement, with its **results** captured in a single **DataSet**. Every run is part of an experiment; most experiments contain many runs. Runs have a (system-assigned counter) number within the experiment.

In [1]:
from new_exp import *

Connect  to a db on disk.

In [2]:
conn = connect(db)

Create an experiment, one can have multiple sample names per experiment (if needed)

In [3]:
exp_name = "majo qbit"
sample = "suspended bridge"
exp_id = new_experiment(conn, exp_name, sample)

Add metadata or really anything to the experiment table

In [4]:
add_meta_data(conn, 1,{"a":1}, "experiments")

##  ParamSpec

Light weight objects that translates the information of a parameter (qcodes) to sql.
Anything can be made into a parameter.

### note 
sqlite is dynamically typed hence the following type declaration does not generate runtime errors if one puts a stirng inside the DB

In [5]:
parameter_a = ParamSpec("a", "INTEGER")
# metadata with key="value", and number=1
parameter_b = ParamSpec("b", "INTEGER", key="value", number=1)

## Run and  DataSet 

A DataSet is the runtime object that maps to a run in the database.

A run in the database is a row in the runs table, and a table with an unique name.

By default we use a `{}-{}-{}` formatter, but we may extend it using the qcodes location thing.


In [6]:
# bare bone completely empty dataset

In [7]:
data_set_id, data_set_name = create_run(conn, last_experiment(conn), "sweep")

In [8]:
# add paraemter 
add_parameter(conn, data_set_name, parameter_b)

In [9]:
import socket
# can add metadata
add_meta_data(conn, data_set_id, {"hostname": socket.gethostname()})

In [10]:
# or all in one
data_set_id, data_set_name = create_run(conn, last_experiment(conn), "sweep",
                                        parameter_a, parameter_b,
                                        metadata={"key": "value"})

In [11]:
# cann add new parameter: an array
parameter_c = ParamSpec("c", "array")
add_parameter(conn, data_set_name, parameter_c)

In [12]:
for _ in range(100):
    insert_values(conn, data_set_name, [parameter_a, parameter_b], [0, 0])
    # moar values
    insert_many_values(conn, data_set_name, [parameter_a], [[1, 2, 3]])
    x = np.arange(100).reshape(10, 10)
    insert_values(conn, data_set_name,
                  [parameter_a, parameter_b, parameter_c], [0, 0, x])

In [13]:
# save!
conn.commit()

In [14]:
# browse

In [15]:
pdb

Automatic pdb calling has been turned ON


In [16]:
# now browse the results
get_data(conn, data_set_name, [parameter_a, parameter_c], 0, 5)

[[0, None],
 [1, None],
 [2, None],
 [3, None],
 [0, array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
         [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
         [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
         [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
         [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
         [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
         [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])]]

In [18]:
# modify result inplace
p = get_parameters(conn, data_set_name)
rows_modified = modify_values(conn, data_set_name, 0, p, [0]*len(p))
print(f"modified {rows_modified} rows")

modified 1 rows


In [21]:
# new dataset!
parameter_a = ParamSpec("x", "INTEGER")
# metadata with key="value", and number=1
parameter_b = ParamSpec("y", "INTEGER", key="value", number=1)
# this would be a new data set
data_set_id, data_set_name = create_run(conn, exp_id, "sweep",
                                        parameter_a, parameter_b,
                                        metadata={"key": "value"})

for i in range(100):
    # moar values
    insert_many_values(conn, data_set_name, [parameter_a], [[1, 2, 3]])

finish_experiment(conn, exp_id)

In [22]:
data_set_name

'sweep-7-5'